In [6]:
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
import os
import urllib.request
import pprint
import json

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}
client_id = os.getenv("NAVER_CLIENT_ID") 
client_secret = os.getenv("NAVER_CLIENT_SECRET")

load_dotenv()

True

In [7]:
base_url = 'https://openapi.naver.com/v1/search/shop.json'
def search_shopping_item(query):
    payload={
        "query":query,
        "display":50,
        "sort":"sim"
    }
    query_string = urllib.parse.urlencode(payload)
    url = f"{base_url}?{query_string}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)

    rescode = response.getcode()


    if(rescode==200):
        response_body=response.read().decode("utf-8")
        result=json.loads(response_body)
        return result["items"]

In [8]:
shopping_item=search_shopping_item("가디건")

In [9]:
def save_to_json(data,name):
    with open(f"data/{name}.json","w",encoding="utf-8") as file:
        json.dump(data,file)

In [10]:
save_to_json(shopping_item,"shops")

## 질문 2 : shops.json 파일을 Pandas DataFrame로 저장하기

In [11]:
import pandas as pd
def return_to_df(filename):
    return pd.read_json(filename)
df=return_to_df("data/shops.json")

## 질문 3 : 검색어로  찾은  Shop의 상품  목록 출력하기

In [15]:
df.head()

,title,link,image,lprice,hprice,mallName,productId,productType,brand,maker,category1,category2,category3,category4
0,백화점3-5일배송 폴로 <b>가디건</b> 걸즈 걸스 랄프로렌 꽈배기 블랙 화이트 ...,https://smartstore.naver.com/main/products/667...,https://shopping-phinf.pstatic.net/main_842166...,64900,,나우인뉴욕,84216650390,2,폴로랄프로렌,폴로랄프로렌,패션의류,여성의류,카디건,
1,봄 가을 간절기 라운드 루즈핏 크롭 <b>가디건</b> 레드 그린 핑크 화이트 블랙...,https://smartstore.naver.com/main/products/546...,https://shopping-phinf.pstatic.net/main_830114...,19900,,ARUMY,83011442056,2,,,패션의류,여성의류,카디건,
2,부드러운 캐시미어 크롭 <b>가디건</b> 숏 라운드 니트<b>가디건</b> 여성 ...,https://smartstore.naver.com/main/products/482...,https://shopping-phinf.pstatic.net/main_823726...,18900,,코이블리,82372678315,2,,,패션의류,여성의류,카디건,
3,[백화점3-5일배송] 폴로 <b>가디건</b> 니트 걸즈 랄프로렌 블랙 화이트 핑크...,https://smartstore.naver.com/main/products/845...,https://shopping-phinf.pstatic.net/main_859949...,63900,,트위티 155,85994962324,2,폴로랄프로렌,폴로랄프로렌,패션의류,여성의류,카디건,
4,[국내발송 + 선물포장] 폴로 랄프로렌 여아 아기 코튼 베이비 <b>가디건</b> ...,https://smartstore.naver.com/main/products/945...,https://shopping-phinf.pstatic.net/main_870040...,69000,,리틀데이즈,87004045841,2,폴로랄프로렌,폴로랄프로렌,출산/육아,유아동의류,카디건,


## 질문 4 :  검색어로  찾은  Shop의 상품  목록 중에서 가격이 50,000원 이하인 상품만 출력하기

In [13]:
cols_to_drop = ["title", "maker", "image", "hprice", "productId",
                "productType", "category1", "category2", "category3", "category4"]
lower_50000 = df[df["lprice"] <= 50000].drop(
    columns=cols_to_drop).sort_values(by="lprice", ascending=True).reset_index(drop=True).loc[:, "lprice":"brand"]

lower_50000.head()

,lprice,mallName,brand
0,17512,네이버,H&M
1,18900,코이블리,
2,18900,코이블리,
3,19800,로이앤한나,
4,19900,ARUMY,


## 질문 5 :  검색어로  찾은  Shop의 상품  목록 중에서 특정 쇼핑몰 상품만 출력하기

In [14]:
from_naver = df[df["mallName"] == "네이버"].sort_values(
    by="lprice", ascending=True).reset_index(drop=True).loc[:, "lprice":"brand"]

from_naver.head()

,lprice,hprice,mallName,productId,productType,brand
0,17512,,네이버,51781610172,1,H&M
1,23000,,네이버,51143733510,1,리스트
2,28070,,네이버,43475137325,1,비비안웨스트우드
3,31416,,네이버,49541497760,1,제너럴아이디어
4,32400,,네이버,53287208769,1,제너럴아이디어
